In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split

import sys


In [2]:
# Import modules from parent directory
sys.path.insert(0,'..')

In [3]:
from ipynb.fs.full.training_preprocessing import GetDataset

FileNotFoundError: [Errno 2] No such file or directory: './Data/xMonths_SimulatedStaff.csv'

In [33]:
df = GetDataset()
df

,Timestamp,subtotal,transaction_count,rain,temperature,holiday,oil_price,workforce_type_1,workforce_type_2,workforce_type_3,workforce_type_4,time_idx,constant_group,year,month,day,hour
0,17/05/2021 10:00,428.03,11.0,0.0,12.7,False,69.62,1.0,2.0,3.0,0.0,0,group_1,2021,5,17,10
1,17/05/2021 11:00,324.78,15.0,0.0,13.4,False,69.62,1.0,2.0,3.0,1.0,1,group_1,2021,5,17,11
2,17/05/2021 12:00,279.75,12.0,0.0,13.6,False,69.62,1.0,2.0,3.0,1.0,2,group_1,2021,5,17,12
3,17/05/2021 13:00,1630.59,29.0,0.0,14.1,False,69.62,1.0,2.0,3.0,1.0,3,group_1,2021,5,17,13
4,17/05/2021 14:00,1166.10,18.0,0.0,15.9,False,69.62,1.0,2.0,3.0,1.0,4,group_1,2021,5,17,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5062,30/11/2022 14:00,4779.11,82.0,0.0,0.0,False,83.45,1.0,4.0,5.0,0.0,5062,group_1,2022,11,30,14
5063,30/11/2022 15:00,2460.84,54.0,0.0,0.0,False,83.45,1.0,4.0,5.0,0.0,5063,group_1,2022,11,30,15
5064,30/11/2022 16:00,2026.30,43.0,0.0,0.0,False,83.45,1.0,4.0,5.0,0.0,5064,group_1,2022,11,30,16
5065,30/11/2022 17:00,2346.13,44.0,0.0,0.0,False,83.45,1.0,4.0,5.0,0.0,5065,group_1,2022,11,30,17


In [ ]:
# Defining custom dataloaders
## train data
class TrainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]

    def __getData__(self):
        return self.X_data, self.y_data
        
    def __len__ (self):
        return len(self.X_data)


train_data = TrainData(torch.FloatTensor(X_train), 
                       torch.FloatTensor(y_train))
## test data    
class TestData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    

test_data = TestData(torch.FloatTensor(X_test))

In [30]:
class MultiLabelRegression(nn.Module):
    def __init__(self, input_size, output_size):
        super(MultiLabelRegression, self).__init__()
        self.linear = nn.Linear(input_size, output_size)
    
    def forward(self, x):
        out = self.linear(x)
        return out


In [ ]:
def train(model, optimizer, criterion, num_epochs, train_loader):
    for epoch in range(num_epochs):
        for inputs, targets in train_loader:
            # Forward pass
            outputs = model(inputs)
            # Compute loss
            loss = criterion(outputs, targets)
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        # Print progress
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))


In [ ]:
# Different type of model to train with test_loader
def train(model, optimizer, criterion, num_epochs, train_loader, test_loader=None):
    for epoch in range(num_epochs):
        train_loss = 0.0
        test_loss = 0.0
        for inputs, targets in train_loader:
            # Forward pass
            outputs = model(inputs)
            # Compute loss
            loss = criterion(outputs, targets)
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        if test_loader:
            with torch.no_grad():
                for inputs, targets in test_loader:
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    test_loss += loss.item()
        # Print progress
        train_loss /= len(train_loader)
        test_loss /= len(test_loader) if test_loader else 1
        print('Epoch [{}/{}], Train Loss: {:.4f}, Test Loss: {:.4f}'.format(epoch+1, num_epochs, train_loss, test_loss))

In [47]:

x = pd.DataFrame(df, columns=['time_idx', 'Timestamp', 'holiday', 'rain', 'temperature', 'hour', 'year', 'month', 'day'])
y = pd.DataFrame(df, columns=['subtotal', 'transaction_count', 'workforce_type_1', 'workforce_type_2', 'workforce_type_3', 'workforce_type_4'])
x

,Timestamp,holiday,rain,temperature,hour,year,month,day
0,17/05/2021 10:00,False,0.0,12.7,10,2021,5,17
1,17/05/2021 11:00,False,0.0,13.4,11,2021,5,17
2,17/05/2021 12:00,False,0.0,13.6,12,2021,5,17
3,17/05/2021 13:00,False,0.0,14.1,13,2021,5,17
4,17/05/2021 14:00,False,0.0,15.9,14,2021,5,17
...,...,...,...,...,...,...,...,...
5062,30/11/2022 14:00,False,0.0,0.0,14,2022,11,30
5063,30/11/2022 15:00,False,0.0,0.0,15,2022,11,30
5064,30/11/2022 16:00,False,0.0,0.0,16,2022,11,30
5065,30/11/2022 17:00,False,0.0,0.0,17,2022,11,30


In [49]:
# Define the data
#x_train = torch.randn(100, 10)  # 100 samples with 10 features each
#y_train = torch.randint(0, 2, (100, 5))  # 100 samples with 5 labels each (0 or 1)
TEST_SIZE = 0.33


X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SIZE)

#X_train = torch.tensor(X_train.values, dtype=torch.float32)
#X_test = torch.tensor(X_test.values, dtype=torch.float32)
#y_train = torch.tensor(y_train.values, dtype=torch.float32)
#y_test = torch.tensor(y_test.values, dtype=torch.float32)


# Define the model
model = MultiLabelRegression(9, 6)

# Define the loss function and the optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters())

# Create the data loader
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=10, shuffle=True)

test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=10, shuffle=True)



TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.